# Saving and loading populations

Another feature Brush implements is the ability to save and load entire populations.
We use JSON notation to store the population into a file that is human readable. The same way, we can feed an estimator a previous population file to serve as starting point for the evolution.

In this notebook, we will walk through how to use the `save_population` and `load_population` parameters. 

We start by getting a sample dataset and splitting it into `X` and `y`:

In [1]:
import pandas as pd
from pybrush import BrushRegressor

# load data
df = pd.read_csv('../examples/datasets/d_enc.csv')
X = df.drop(columns='label')
y = df['label']

To save the population after finishing the evolution, you nee to set `save_population` parameter to a value different than an empty string. Then, the final population is going to be stored in that specific file.

In this example, we create a temporary file.

In [2]:
import pickle
import os, tempfile

pop_file = os.path.join(tempfile.mkdtemp(), 'population.json')

# set verbosity==2 to see the full report
est = BrushRegressor(
    functions=['SplitBest','Add','Mul','Sin','Cos','Exp','Logabs'],
    max_gens=10,
    objectives=["scorer", "complexity"],
    scorer='mse',
    save_population=pop_file,
    use_arch=True, # Only the pareto front of last gen will be stored in archive
    verbosity=2
)

est.fit(X,y)
y_pred = est.predict(X)
print('score:', est.score(X,y))

Generation 1/10 [//////                                            ]
Best model on Val:If(x0>0.75,If(x0>0.81,7.05*Add(6.40*Cos(-6.10*Logabs(11.47*Sin(1.01*Add(1.01*x3,1.20*x4)))),-3.64*Sin(0.90*x3)),If(x6>0.32,0.41*Add(0.52*x3,0.08*x2),1.91*Mul(1.89*x0,1.89*x4))),If(x0>0.70,0.10*Mul(0.03*x3,0.09*x3),If(x0>0.63,17.11,24.59*x0)))
Train Loss (Med): 8.30766 (81.98991)
Val Loss (Med): 8.30766 (81.98991)
Median Size (Max): 7 (138)
Median complexity (Max): 314 (848846280)
Time (s): 0.90195

Generation 2/10 [///////////                                       ]
Best model on Val:If(x0>0.75,If(x0>0.81,2.91*Add(2.63*Cos(If(x6>0.17,If(x0>0.84,0.69*x2,30.74*x4),0.62*x2)),20.44*Sin(0.17*x3)),If(x6>0.32,0.05*Add(4.57*x3,0.70*x2),7.74*Mul(0.39*x0,2.28*x4))),If(x0>0.70,0.02*Mul(0.13*x3,0.13*x3),If(x0>0.63,17.11,24.59*x0)))
Train Loss (Med): 7.33267 (60.79966)
Val Loss (Med): 7.33267 (60.79966)
Median Size (Max): 7 (106)
Median complexity (Max): 24 (1630685352)
Time (s): 1.38966

Generation 3/10 [///////

Loading a previous population is done providing `load_population` a string value corresponding to a JSON file generated by Brush. In our case, we will use the same file from the previous code block.

After loading the population, we run the evolution for 10 more generations, and we can see that the first generation started from the previous population. This means that the population was successfully saved and loaded.

In [3]:
est = BrushRegressor(
    functions=['SplitBest','Add','Mul','Sin','Cos','Exp','Logabs'],
    load_population=pop_file,
    max_gens=10,
    verbosity=1
)

est.fit(X,y)
y_pred = est.predict(X)
print('score:', est.score(X,y))

Loaded population from /var/folders/mh/ggnb_jv93cl_gbqbg2jxq0yr0000gn/T/tmpx55nrr7z/population.json of size = 200
Completed 100% [====================]
saving final population as archive...
score: 0.9374115013075351


There is a convenient way of accessing individuals on the population: just use the index of the individual on the `est.population_` list.

In [4]:
# it will contain all individuals, differently than the archive
print("population size:", len(est.population_))
print("archive size   :", len(est.archive_))

est.population_[0]

population size: 100
archive size   : 7


{'fitness': {'complexity': 2,
  'crowding_dist': 3.4028234663852886e+38,
  'dcounter': 0,
  'depth': 1,
  'dominated': [1, 6],
  'linear_complexity': 2,
  'loss': 90.3851318359375,
  'loss_v': 90.3851318359375,
  'prev_complexity': 2,
  'prev_depth': 1,
  'prev_linear_complexity': 2,
  'prev_loss': 90.3851318359375,
  'prev_loss_v': 90.3851318359375,
  'prev_size': 1,
  'rank': 1,
  'size': 1,
  'values': [90.3851318359375, 2.0],
  'weights': [-1.0, -1.0],
  'wvalues': [-90.3851318359375, -2.0]},
 'id': 221,
 'is_fitted_': False,
 'objectives': ['mse', 'linear_complexity'],
 'parent_id': [262],
 'program': {'Tree': [{'W': 24.585369110107422,
    'arg_types': [],
    'center_op': True,
    'feature': 'constF',
    'feature_type': 'ArrayF',
    'fixed': False,
    'is_weighted': True,
    'name': 'Constant',
    'node_type': 'Constant',
    'prob_change': 0.6167147755622864,
    'ret_type': 'ArrayF',
    'sig_dual_hash': 7018942542468397869,
    'sig_hash': 14162902253047951597}],
  'is_

you can convert the json representation back to an fully functional individual by wrapping it in the individual class. It is important that the type of individual (i.e. classification, regression) is the same.

Differently from the archive (which is sorted by complexity), the individuals in the population have no specific order. So individual 5 may or may not be more complex than individual 10, for example.

In [5]:
from pybrush import individual

loaded_from_population = individual.RegressorIndividual.from_json(est.population_[2])
print(loaded_from_population.get_model("tree"))

Add
|- 24.35
|- x6


## Saving just the archive

In case you want to use another expression rather than the final `best_estimator_`, brush provides the archive option.

The archive is just the pareto front from the population. You can use `predict_archive` (and `predict_proba_archive` if using a `BrushClassifier`) to call the prediction methods for the entire archive, instead of the selected best individual.

But first, you need to enable this option with `use_arch=True`. When set to False, it will store the entire final population

In [6]:
est = BrushRegressor(
    functions=['SplitBest','Add','Mul','Sin','Cos','Exp','Logabs'],
    load_population=pop_file,
    use_arch=True,
    max_gens=10,
    verbosity=1
)

est.fit(X,y)

# accessing first expression from the archive. It is serialized as a dict
print(est.archive_[0]['fitness'])

Loaded population from /var/folders/mh/ggnb_jv93cl_gbqbg2jxq0yr0000gn/T/tmpx55nrr7z/population.json of size = 200
Completed 100% [====================]
{'complexity': 2, 'crowding_dist': 0.0, 'dcounter': 0, 'depth': 1, 'dominated': [], 'linear_complexity': 2, 'loss': 90.3851318359375, 'loss_v': 90.3851318359375, 'prev_complexity': 2, 'prev_depth': 1, 'prev_linear_complexity': 2, 'prev_loss': 90.3851318359375, 'prev_loss_v': 90.3851318359375, 'prev_size': 1, 'rank': 1, 'size': 1, 'values': [90.3851318359375, 2.0], 'weights': [-1.0, -1.0], 'wvalues': [-90.3851318359375, -2.0]}


You can open the serialized file and change individuals' programs manually.

This also allow us to have checkpoints in the execution.

## Using population files with classification

To give another example, we do a two-step fit in the cells below.

First, we run the evolution and save the population to a file; then, we load it and keep evolving the individuals.

What is different though is that the first run is optimizing `scorer` and `complexity`, and the second run is optimizing `average_precision_score` and `linear_complexity`.

In [7]:
from pybrush import BrushClassifier

# load data
df = pd.read_csv('../examples/datasets/d_analcatdata_aids.csv')
X = df.drop(columns='target')
y = df['target']

pop_file = os.path.join(tempfile.mkdtemp(), 'population.json')

est = BrushClassifier(
    functions=['SplitBest','Add','Mul','Sin','Cos','Exp','Logabs'],
    max_gens=10,
    objectives=["scorer", "complexity"],
    scorer="log",
    save_population=pop_file,
    pop_size=200,
    verbosity=2
)

est.fit(X,y)

print("Best model:", est.best_estimator_.get_model())
print('score:', est.score(X,y))

Generation 1/10 [//////                                            ]
Best model on Val:Logistic(Sum(-0.39,If(AIDS>15890.50,0.38*Logabs(If(AIDS>18390.00,If(AIDS>22304.00,7.87*AIDS,8.15*AIDS),31.07*Sin(22304.00*AIDS))),0.04*Logabs(If(Total>1572255.50,0.01*Sin(18390.89),62.10*Mul(62.10*AIDS,62.10*AIDS))))))
Train Loss (Med): 0.52166 (2.86439)
Val Loss (Med): 0.52166 (2.86439)
Median Size (Max): 10 (114)
Median complexity (Max): 992 (2014221728)
Time (s): 0.71757

Generation 2/10 [///////////                                       ]
Best model on Val:Logistic(Sum(-0.39,If(AIDS>15890.50,Logabs(If(AIDS>18390.00,If(AIDS>22304.00,7.87*AIDS,8.15*AIDS),31.07*Sin(22304.00*AIDS))),0.04*Logabs(If(Total>1572255.50,0.01*Sin(18390.89),62.10*Mul(62.10*AIDS,62.10*AIDS))))))
Train Loss (Med): 0.51282 (0.69315)
Val Loss (Med): 0.51282 (0.69315)
Median Size (Max): 10 (166)
Median complexity (Max): 752 (1777037216)
Time (s): 1.23145

Generation 3/10 [////////////////                                  ]
Best m

In [8]:
from sklearn.metrics import accuracy_score

accuracy_score(y, est.predict(X))

0.84

In [9]:
est = BrushClassifier(
    functions=['SplitBest','Add','Mul','Sin','Cos','Exp','Logabs'],
    load_population=pop_file,
    objectives=["scorer", "complexity"],
    scorer="average_precision_score",
    max_gens=10,
    pop_size=200, # make sure this is the same as loaded pop
    use_arch=True,
    verbosity=1
)

est.fit(X,y)

print("Best model:", est.best_estimator_.get_model())
print('score:', est.score(X,y))

Loaded population from /var/folders/mh/ggnb_jv93cl_gbqbg2jxq0yr0000gn/T/tmpqguuq9uj/population.json of size = 400
Completed 100% [====================]
Best model: Logistic(Sum(If(AIDS>15890.50,18391.79*Logabs(If(AIDS>18390.00,If(AIDS>22304.00,8.94*AIDS,8.55*AIDS),-4.58*Cos(1.30*Sin(If(AIDS>1.00,22304.00*AIDS,-3.15*AIDS))))),1.29*Logabs(-1759.52*Mul(-1640.28,18389.99)))))
score: 0.5


We can see the fitness object, and that the scorer now matches the average precision score metric:

In [10]:
# Fitness is (scorer, linear complexity)
print(est.best_estimator_.fitness)

Fitness(1.022064 4.000000 )


In [11]:
from sklearn.metrics import average_precision_score

# takes y_true as first argument, and y_pred as second argument.
average_precision_score(y, est.predict_proba(X)[:, 1]) #, average='weighted')

0.5

## Serialization with pickle

You can save the entire model (best individual, parameters, and archive) with pickle. 

> At the current stage, Brush does not serialize the search space and dataset references, but only the necessary information to be able to load a previously trained model and do predictions with it.

In [12]:
est

,pop_size,200
,max_gens,10
,max_stall,0
,max_time,-1
,verbosity,1
,algorithm,'nsga2'
,mode,'classification'
,max_depth,10
,max_size,100
,num_islands,5
,mig_prob,0.05


In [13]:
import pickle

est_file = os.path.join(tempfile.mkdtemp(), 'est.pkl')

with open(est_file, 'wb') as f:
    pickle.dump(est, f)

In [14]:
loaded_est = pickle.load(open(est_file, 'rb'))

In [15]:
print(est.predict(X))
print(loaded_est.predict(X))

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True]


In [16]:
print(est.predict_archive(X)[0])
print(loaded_est.predict_archive(X)[0])

{'id': 472, 'y_pred': array([False,  True,  True,  True,  True, False,  True,  True,  True,
       False, False,  True,  True, False, False, False, False, False,
       False, False,  True, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True,  True, False,  True, False,
       False, False,  True,  True, False])}
{'id': 472, 'y_pred': array([False,  True,  True,  True,  True, False,  True,  True,  True,
       False, False,  True,  True, False, False, False, False, False,
       False, False,  True, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True,  True, False,  True, False,
       False, False,  True,  True, False])}


## Stop/resume the fitting of an estimator

In the code below I try to mimic how pytorch models are trained: we can stop the training at any time, and we can resume it later. 

The idea is to demonstrate how to use population files to store checkpoints, and continuing from the last saved checkpoint.

In [17]:
def train(est, X, y):
    checkpoint = os.path.join(tempfile.mkdtemp(), 'brush_pop_checkpoint.json')
    
    step = 5
    max_gens = est.max_gens
    est.max_gens = step
    est.save_population = checkpoint
    est.load_population = ""
    
    # You can set validation_size to a value greater than zero
    # and shuffle_split to true to have random bathes of data
    est.shuffle_split = True
    est.validation_size = 0.2
    
    for g in range(max_gens // step):
        print(f"Progress {g + 1}/{max_gens // step}")
        
        est.fit(X, y) # Notice that this will reset the MAB everytime!

        # Enable loading the checkpoint after a first run
        est.load_population = checkpoint
        
        print("Best model:", est.best_estimator_.get_model())
        print('score     :', est.score(X, y))

    # Restoring initial state
    est.max_gens = max_gens

In [18]:
est = BrushClassifier(
    objectives=["scorer", "linear_complexity"],
    scorer="balanced_accuracy",
    max_gens=50,
    validation_size=0.2,
    pop_size=100,
    max_depth=20,
    max_size=50,
    verbosity=1
)

train(est, X, y)

Progress 1/10
Completed 100% [====================]
saving final population as archive...
Saved population to file /var/folders/mh/ggnb_jv93cl_gbqbg2jxq0yr0000gn/T/tmpmzzid_sc/brush_pop_checkpoint.json
Best model: Logistic(Sum(0.06,0.57*Cos(1.00*Mean(1.51,1.00*Mean(AIDS,3.04,1.00*AIDS,1.00*Prod(1.00*Sqrtabs(AIDS),AIDS,AIDS)),2.37))))
score     : 0.64
Progress 2/10
Loaded population from /var/folders/mh/ggnb_jv93cl_gbqbg2jxq0yr0000gn/T/tmpmzzid_sc/brush_pop_checkpoint.json of size = 200
Completed 100% [====================]
saving final population as archive...
Saved population to file /var/folders/mh/ggnb_jv93cl_gbqbg2jxq0yr0000gn/T/tmpmzzid_sc/brush_pop_checkpoint.json
Best model: Logistic(Sum(0.12,Cos(1.00*Mean(1.95,1.00*Mean(AIDS,13.92,AIDS,1.00*Prod(AIDS,AIDS,AIDS)),4.89))))
score     : 0.72
Progress 3/10
Loaded population from /var/folders/mh/ggnb_jv93cl_gbqbg2jxq0yr0000gn/T/tmpmzzid_sc/brush_pop_checkpoint.json of size = 200
Completed 100% [====================]
saving final popu

By default, sklearn estimators will reset when calling fit twice. To continue from last fit, you can call `partial_fit` and brush will resume the training.

If you want, you can change parameters from the `est` object before calling partial fit to update the execution settings.

It is important that the data has the same features (same name and dtype) as the data used in the previous `fit`/`partial_fit`.

In [19]:
print(est.best_estimator_.get_model())
print(est.best_estimator_.fitness)

est.partial_fit(X, y)

print(est.best_estimator_.get_model())
print(est.best_estimator_.fitness)

Logistic(Sum(-130.08,AIDS))
Fitness(0.600000 16.000000 )
Completed 100% [====================]
saving final population as archive...
Saved population to file /var/folders/mh/ggnb_jv93cl_gbqbg2jxq0yr0000gn/T/tmpmzzid_sc/brush_pop_checkpoint.json
Logistic(Sum(-130.08,1.00*Sqrt(AIDS)))
Fitness(0.900000 21.000000 )


The `partial_fit` also allows you to fix an initial portion of the tree before doing the new fit.

You can also choose to leave leaves out of this locking mechanism, this way the terminals close to the root are unlocked and can change.

If you set a big depth and also force leaves to be locked, there may be some (smaller) programs in the population that will not change at all during the run.

In [20]:
print(est.best_estimator_.get_model())
print(est.best_estimator_.fitness)

est.partial_fit(X, y, lock_nodes_depth=2, skip_leaves=True)

print(est.best_estimator_.get_model())
print(est.best_estimator_.fitness)

Logistic(Sum(-130.08,1.00*Sqrt(AIDS)))
Fitness(0.900000 21.000000 )
Completed 100% [====================]
saving final population as archive...
Saved population to file /var/folders/mh/ggnb_jv93cl_gbqbg2jxq0yr0000gn/T/tmpmzzid_sc/brush_pop_checkpoint.json
Logistic(Sum(-130.08,AIDS))
Fitness(0.600000 16.000000 )
